# Introduction to Machine Learning, UZH 2018, Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
##       
##    
# 4. Collection of all Pipes

###        
In this section we use the data we prepared in chapter 1.  



In [1]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC
plt.style.use('seaborn-whitegrid')

###    
###    
## 4.1. Pipe 1 (P1)

#### Feature Selection: In-Pipe with RandomForestClassifier or PCA
#### Scaling : In-Pipe with StandardScaler
#### Classification: SVM
#### Additional Classification: RandomForestClassifier, LogisticRegression
###      

### 4.1.0.  (i) DATA SETTINGS (P1)


### Choose the Dataset Version you want
##### Whole Feature Matrices (Features not pre-selected)
VERSION = 1; Feature Matrix with only ratios                                  
VERSION = 2;  Feature Matrix with ratios + saisonality + other market data



In [2]:
### Chose which dataset version you want the selection of features and the prediction to be based on 
VERSION = 1
"""
INSERT NUMBER 1 or 2
"""

# Defining sel_state variable for easier printing out    
if VERSION == 1: sel_version = 'Whole original Dataset with only the Ratios as predicive Features.'
elif VERSION == 2: sel_version = 'Whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
else: raise ValueError('VERSION must be either 1 or 2')

In [3]:
# print status
briefing_data='SUMMARY DATA SETTINGS:'+'\n'+'Selected Version ' + str(VERSION) + ': ' + str(sel_version)+'\n'
print(briefing_data)

SUMMARY DATA SETTINGS:
Selected Version 1: Whole original Dataset with only the Ratios as predicive Features.



##### Import necessary Data and impute

In [4]:
# Import Data
if VERSION == 1: 
# features not pre-selected, only ratios
    X = pd.read_csv('Data/generated_datasets/features_ratios_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 2: 
# features not pre-selected, ratios + seasonality + market data
    X = pd.read_csv('Data/generated_datasets/features_additional_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1 or 2')   

In [5]:
# Train/test split, into 20% test size and 80% train size because it is a relatively small dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Use a median fill for train
imp = Imputer(missing_values=np.nan, strategy = 'median' , axis=0)
imputed_dataset = pd.DataFrame(imp.fit_transform(X_train))
imputed_dataset.columns = X_train.columns
imputed_dataset.index = X_train.index
X_train = imputed_dataset

# Use a median fill for the test set
imputed_dataset = pd.DataFrame(imp.fit_transform(X_test))
imputed_dataset.columns = X_test.columns
imputed_dataset.index = X_test.index
X_test = imputed_dataset

### 4.2.0. (ii) PIPING SETTINGS (P1)

In [6]:
# Set Kernel
set_kernel = 'rbf'
"""
Select 'poly', 'linear' or 'rbf'
"""

# Choose In-Pipe Feature Selection Method
set_feature_selecter = RandomForestClassifier
"""
Select RandomForestClassifier or PCA
"""

# Choose Classifier
set_classifier = LogisticRegression
"""
Select LogisticRegression or RandomForestClassifier
"""

# Print result of settings
briefing_pipe='SUMMARY PIPE SETTINGS:'+'\n'+'Selected kernel: '+str(set_kernel)+'.'+'\n'+'Selected In-Pipe Feature Selecter: '+str(set_feature_selecter)+'.'+'\n'+'Additional Classifier: '+str(set_classifier)+'.'+'\n'
print(briefing_pipe)

SUMMARY PIPE SETTINGS:
Selected kernel: rbf.
Selected In-Pipe Feature Selecter: <class 'sklearn.ensemble.forest.RandomForestClassifier'>.
Additional Classifier: <class 'sklearn.linear_model.logistic.LogisticRegression'>.



### 4.2.1. Piping (P1)

In [13]:
# set conditions so according to choices under "settings", the methods are going to be used automatically

# For Loop defining paremeters for Feature Selecter
if set_feature_selecter == PCA: 
    feat_sel_function = PCA()
    fs_attr1  ='fs__n_components'
    fs_attr1_par = [8, 10, 15, 20, 30, 40]
    fs_attr2 = 'fs__random_state'
    fs_attr2_par = [0, 10, 20, 50, 100]  

elif set_feature_selecter == RandomForestClassifier: 
    feat_sel_function = SelectFromModel(estimator=RandomForestClassifier(), threshold = 'median')
    fs_attr1 = 'fs__estimator__n_estimators'
    fs_attr1_par = [3, 6, 8, 10, 15, 20, 30, 40]
    fs_attr2 = 'fs__estimator__random_state'
    fs_attr2_par = [0, 10, 20, 50, 100]
else: raise ValueError('feature_selecter must be either PCA or RandomForestClassifier')

# For Loop defining paremeters for additional Classifier
if set_classifier == RandomForestClassifier: 
    classifier_function = RandomForestClassifier()
    cl_attr1 = 'cl__n_estimators'
    cl_attr1_par = [6, 8, 10, 15, 20, 30, 40]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par = [0, 10, 20, 50, 100]
elif set_classifier == LogisticRegression: 
    classifier_function = LogisticRegression()
    cl_attr1 = 'cl__C'
    cl_attr1_par = [1, 10, 100]
    cl_attr2 = 'cl__random_state'
    cl_attr2_par = [0, 10, 20, 50, 100]
else: raise ValueError('feature_selecter must be either PCA or RandomForestClassifier')

#display(fs_attr1, fs_attr1_par)
#display(fs_attr2, fs_attr2_par)
#display(cl_attr1, cl_attr1_par)
#display(cl_attr2, cl_attr2_par)

# print settings
print(briefing_data+'\n'+briefing_pipe)

SUMMARY DATA SETTINGS:
Selected Version 1: Whole original Dataset with only the Ratios as predicive Features.

SUMMARY PIPE SETTINGS:
Selected kernel: rbf.
Selected In-Pipe Feature Selecter: <class 'sklearn.ensemble.forest.RandomForestClassifier'>.
Additional Classifier: <class 'sklearn.linear_model.logistic.LogisticRegression'>.



In [14]:
# Create pipeline object with standard scaler, chosen Feature Selecter and SVC estimator
# Standardscaler standardizes the input variables
pipe1_a = Pipeline([('scaler', StandardScaler()), 
                  ('fs', feat_sel_function), 
                  ('cl', SVC(kernel=set_kernel, random_state=0))])
pipe1_b = Pipeline([('scaler', StandardScaler()), 
                  ('fs', feat_sel_function), 
                  ('cl', classifier_function)])

# Display scores from resulting pipes
display(pipe1_a.fit(X_train, y_train).score(X_test, y_test))
display(pipe1_b.fit(X_train, y_train).score(X_test, y_test))


0.6225352112676056

0.6225352112676056

In [ ]:
# Define parameter grid

# First part: scaling with StandardScaler, 
# Feature Selection with selected method, 
# Classification with SVC with selected kernel

# Second part:  scaling with StandardScaler,
# Feature Selection with selected method, 
# Classification with selected 'additional classifier' method

param_grid1 = [{'scaler': [StandardScaler()],
               'fs': [feat_sel_function],
                fs_attr1: fs_attr1_par,
                fs_attr2: fs_attr2_par,                
               'cl': [SVC(kernel=set_kernel)],
               'cl__C': [10, 100]},
               {'scaler': [StandardScaler()],
                'fs': [feat_sel_function],
                cl_attr1: cl_attr1_par,
                cl_attr2: cl_attr2_par, 
                'cl': [classifier_function],
                cl_attr1: cl_attr1_par,
                cl_attr2: cl_attr2_par}]

In [ ]:
# Grid Search
grid1_a = GridSearchCV(pipe1_a, param_grid1, cv=5, n_jobs=-1)
grid1_a.fit(X_train, y_train)




In [ ]:
grid1_b = GridSearchCV(pipe1_b, param_grid1, cv=5, n_jobs=-1)
grid1_b.fit(X_train, y_train)

In [ ]:



# Print results
#print('Best CV accuracy: {:.2f}'.format(grid.best_score_))
#print('Test score:       {:.2f}'.format(grid.score(X_test_bal, y_test_bal)))
#print('Best parameters: {}'.format(grid.best_params_))

###    
###    
## 4.3. Pipe 2: 
#### Feature Selection:  before Pipe with RandomForestClassifier or PCA (Chapter 2.A and 2.B)
#### Scaling : In-Pipe with StandardScaler
#### Classification: SVM
#### additional Classification: RandomForestClassifier, LogisticRegression
###      

###    
###    
## 4.3. Pipe 3: 
### Feature Selection:  none
### Classification: SVM
### additional Classification: RandomForestClassifier, LogisticRegression